# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686547


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:09,  2.48s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:02,  2.33s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:45,  1.77s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:34,  1.38s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:16<00:01,  2.51it/s]

Rendering models:  93%|█████████▎| 28/30 [00:17<00:01,  2.00it/s]

Rendering models:  97%|█████████▋| 29/30 [00:18<00:00,  1.63it/s]

Rendering models: 100%|██████████| 30/30 [00:18<00:00,  1.78it/s]

AlexMercer                            0.000609
not-logged-in-a9247261537fa1da33fe    0.380777
shirogane                             0.000218
jamescross20                          0.000370
not-logged-in-f7e2260ed4a9517a3e4a    0.000256
kjkavene                              0.000287
rnevils                               0.000304
jmfranci                              0.001941
mbaker23                              0.000595
not-logged-in-ef704c739eb8412f355e    0.003155
sn346808                              0.023350
Rmarks1701                            0.000389
ToriPizza                             0.003027
LeeJoseph                             0.006869
KrisKeogh                             0.010669
Sylverone                             0.000546
not-logged-in-120167733761deee9007    0.000267
ayluminati                            0.000865
ovincent                              0.000348
Carlos5                               0.007423
ControlledChaos                       0.000423
CromarFutureG

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())